# AI Research - Phishing Detection

## Import

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import svm
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
import wordcloud
import matplotlib.pyplot as plt
import numpy as np
import re
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from langdetect import detect
from sklearn.model_selection import GridSearchCV
import enchant 

Shortcut:

In [31]:
# Read cleaned Data from CSV
df_train = pd.read_csv("Datasets/cleaned/train.csv")
df_valid = pd.read_csv("Datasets/cleaned/validation.csv")
df_test = pd.read_csv("Datasets/cleaned/test.csv")
df_gpt = pd.read_csv("Datasets/cleaned/gpt.csv")
df = pd.read_csv("Datasets/cleaned/spam.csv")

The datasets we work with in the further steps are read in. 

In [14]:
df = pd.read_csv("Datasets/spam2.csv")
df_train_roh = pd.read_csv("Datasets/train.csv")
df_test_roh = pd.read_csv("Datasets/test.csv")
df_valid_roh = pd.read_csv("Datasets/validation.csv")
df_gpt = pd.read_csv("df_gpt.csv")

In the preprocessing step we lemmatize the dataset and remove stopwords. Therefore we use NLTK.

In [3]:
nltk.download ("wordnet")
nltk.download ("stopwords")
stopWords = set(stopwords.words('english'))
regexp = RegexpTokenizer('\w+')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Tim\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Tim\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Clean up old Dataframe

In [16]:
# Rename the Columns to one uniform format
df_train_roh = df_train_roh.rename(columns={"sentence1": "text"})
df_valid_roh = df_valid_roh.rename(columns={"sentence1": "text"})
df_test_roh = df_test_roh.rename(columns={"sentence1": "text"})
df = df.rename(columns={"v2": "text", "v1": "label"})

# Replace Labels
df.label = df.label.str.replace("ham", "normal")

# Drop NaN values
df = df.loc[np.logical_and(df.label.notnull(), df.text.notnull())]
df_train_roh = df_train_roh.loc[np.logical_and(df_train_roh.label.notnull(), df_train_roh.text.notnull())]
df_valid_roh = df_valid_roh.loc[np.logical_and(df_valid_roh.label.notnull(), df_valid_roh.text.notnull())]
df_test_roh = df_test_roh.loc[np.logical_and(df_test_roh.label.notnull(), df_test_roh.text.notnull())]

# Drop Columns
df_train_roh.drop("id", axis=1, inplace=True)
df_valid_roh.drop("id", axis=1, inplace=True)
df_test_roh.drop("id", axis=1, inplace=True)
df = df[["text", "label"]]

### Import GPT generated test Mails

## Preprocessing

To prepare our data to train our model, we go through typical preprocessing steps. Therefore we create a lemmatize function, which we can apply on the different datasets. In this part the data is also upsampled, so that tere is the same amount of spam mails and normal mails in the dataset.

In [20]:
dic = enchant.Dict("en_US")

In [21]:
def is_english(text):
    try:
        if detect(text) == "en":
            return 1
        else:
            return 0
    except:
        return 0

wnl = WordNetLemmatizer()
def Lemmatize(x):
    x = regexp.tokenize(x)
    text = ""
    for i in x:
        if i not in stopWords and dic.check(i):
            #lemm = wnl.lemmatize(i)
            text += i + " "
    return text

In [22]:
# Daten mit label "normal" herausfiltern
df_train_normal = df_train_roh.loc[df_train_roh.label == "normal"].copy()
df_valid_normal = df_valid_roh.loc[df_valid_roh.label == "normal"].copy()
df_test_normal = df_test_roh.loc[df_test_roh.label == "normal"].copy()

In [23]:
# Selbe Menge an Daten mit label "spam" herausfiltern
df_train_spam = df_train_roh.loc[df_train_roh.label == "spam"].sample(len(df_train_normal))
df_valid_spam = df_valid_roh.loc[df_valid_roh.label == "spam"].sample(len(df_valid_normal))
df_test_spam = df_test_roh.loc[df_test_roh.label == "spam"].sample(len(df_test_normal))

In [24]:
# Daten zusammenführen
df_train = pd.concat([df_test_normal, df_train_spam])
df_valid = pd.concat([df_valid_normal, df_valid_spam])
df_test = pd.concat([df_train_normal, df_test_spam])

In [26]:
# Daten mischen
df_train = df_train.sample(frac=1)
df_valid = df_valid.sample(frac=1)
df_test = df_test.sample(frac=1)

In [25]:
# Daten vorbereiten
df_train["text"] = df_train.text.apply(Lemmatize)
df_valid["text"] = df_valid.text.apply(Lemmatize)
df_test["text"] = df_test.text.apply(Lemmatize)
df_gpt.text = df_gpt.text.apply(Lemmatize)
df.text = df.text.apply(Lemmatize)

In [201]:
sample = df.sample(int(round(len(df)/2,0)))
df_train2 = df_train_roh.loc[df_train_roh.label == "normal"]

In [205]:
df_train2 = pd.concat([sample, df_train2])

In [209]:
diff = len(df_train2.loc[df_train2.label == "normal"]) - len(df_train2.loc[df_train2.label == "spam"])

In [210]:
df_train2_spam = df_train_roh.loc[df_train_roh.label == "spam"].sample(diff)

In [214]:
df_train2 = pd.concat([df_train2, df_train2_spam])

In [215]:
df_train2.label.value_counts()

normal    4999
spam      4999
Name: label, dtype: int64

In [216]:
df_train2.text = df_train2.text.apply(Lemmatize)

In [217]:
df_train2

,text,label
4915,Just dropped em back,normal
933,What time think Need know I near campus,normal
3797,Feb gt I LOVE U day Send dis VALUED If 3 comes...,normal
2561,And maybe pressies,normal
239,U 447801259231 secret admirer looking 2 make c...,spam
...,...,...
50236,bit,spam
51348,Groovy explorer mind finding young e girl My n...,spam
115810,Volume 2 Issue 10 August 1858 1 Charles mere c...,spam
93577,Said would make sense cried charlie waited pri...,spam


In [ ]:
df_train["english"] = df_train.text.apply(is_english)
df_test["english"] = df_test.text.apply(is_english)
df["english"] = df.text.apply(is_english)

In [28]:
df_train = df_train.loc[df_train.english == 1]
df_test = df_test.loc[df_test.english == 1]
df = df.loc[df.english == 1]

In [26]:
df_train.text = df_train.text.apply(Lemmatize)

C:\Users\tim-h\AppData\Local\Temp\ipykernel_13276\1897205628.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train.text = df_train.text.apply(Lemmatize)


We lemmatize the spam2 dataset and create the columns "text" and "label". The "text" column contains the text of the mail and the "label" column contains the label, if the text is spam or ham. 

In [30]:
#df_train.text = df_train.text.transform(Lemmatize)
df_test.text = df_test.text.transform(Lemmatize)
df.text = df.text.transform(Lemmatize)

### train & test datasets

In the following steps we prepare the data of the train and test dataset. The data comes from one dataset, which we split into training and test data. Therefore we rename the columns, remove a column named "id", that is not important for the next steps, and check if the datasets contains data that is not available. 

We searched for further features, except of the text itself, that we can use to train the model. Therefore we create to functions to see if the text contains links and ip adresses. With the following function we check if the the texts contain links. We want to create a new column with this information, so we can use it to train the model later on. We also create a column with the lenght of the text. 

In [31]:
def containslink(text):
  pattern = r"(http|ftp|https)://([\w-]+(?:(?:.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?"
  return int(bool(re.search(pattern, text)))

df_train['contains_link'] = df_train['text'].apply(containslink)
df_test['contains_link'] = df_test['text'].apply(containslink)


C:\Users\tim-h\AppData\Local\Temp\ipykernel_13276\3454260033.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train['contains_link'] = df_train['text'].apply(containslink)


In [32]:
def containsip(text):
  pattern = r"\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}"
  return int(bool(re.search(pattern, text)))

df_train['contains_ip'] = df_train['text'].apply(containsip)
df_test['contains_ip'] = df_test['text'].apply(containsip)

C:\Users\tim-h\AppData\Local\Temp\ipykernel_13276\2596868674.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train['contains_ip'] = df_train['text'].apply(containsip)


In [33]:
df_train['length'] = df_train['text'].apply(len)
df_test['length'] = df_test['text'].apply(len)

C:\Users\tim-h\AppData\Local\Temp\ipykernel_13276\4249468257.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train['length'] = df_train['text'].apply(len)


In [34]:
df_train

,text,label,english,contains_link,contains_ip,length
0,Free shipping jewelry accessories orders today...,spam,1,0,0,66
2,U Room may anything Chad anyone would somethin...,spam,1,0,0,827
3,Fantastic luxury items less half price Want Di...,spam,1,0,0,59
4,Dreams achievable websites like let enjoy life...,spam,1,0,0,76
5,place Well standing She engineered teeth detai...,spam,1,0,0,1416
...,...,...,...,...,...,...
1503,PUBLIC 1 0 Transitional EN org TR transitional...,spam,1,0,0,6192
1504,We help recharge health skull,spam,1,0,0,30
1505,Any man last 40 minutes MEN S JOURNAL Health P...,spam,1,0,0,1969
1506,M hate It decision entirely blasphemy meriting...,spam,1,0,0,2885


## Models 

We train two different models to see which of them performs better. Therefore we use the Support Vector Machine and the Naive Bayes Classifier.

### SVM - spam2 dataset

In [13]:
cv = CountVectorizer()
features = cv.fit_transform(df.text)

model = svm.SVC()
model.fit(features, df.label)

SVC()

Test if the model detect our testMail.txt as a spam mail:

In [14]:
f = open("testMail.txt", "r")

features_test = cv.transform(f)
# print(model.score(features_test,y_test))
print(model.predict(features_test))

['normal']


In [15]:
features_test = cv.transform(df.text)
print(model.score(features_test,df.label))

0.9967695620961953


The SVM model, that we trained with the spam2 dataset, has an accuracy of 0.99.

### SVM - train dataset

Training the model with the train dataset:

In [17]:
cv = CountVectorizer()
features = cv.fit_transform(df_train.iloc[0:10000].text)

In [18]:
model = svm.SVC(degree=1)
model.fit(features,df_train.iloc[0:10000].label)

SVC(degree=1)

In [19]:
features_test = cv.transform(df_test.iloc[0:10000].text)
print(model.score(features_test,df_test.iloc[0:10000].label))

0.987


We test the model with the test dataset and get an accuracy of 0.98.

Training the model with the test data to compare if it has a better accuracy.

In [16]:
df_test = df_test.loc[np.logical_and(df_test.label.notnull(), df_test.text.notnull())]

features_test = cv.transform(df_test.text)
print(model.score(features_test,df_test.label))

0.04664433451485997


The testmail is now detected as spam:

In [20]:
f = open("testMail.txt", "r")

features_test = cv.transform(f)
# print(model.score(features_test,y_test))
print(model.predict(features_test))

['spam']


### Gaussian Naive Bayes Classifier

We limit our the test and training dataset by using only the first 2000 lines for training because of the size of the dataset and therefore resulting performance issues. 
We train the model with the **train** data.

In [227]:
cv = CountVectorizer()
features = cv.fit_transform(df_train.iloc[0:3500].text).toarray()
valid_features = cv.transform(df_valid.iloc[0:3500].text).toarray()
test_features = cv.transform(df_test.iloc[0:3500].text).toarray()

In [233]:
gnb = GaussianNB()
gnb = gnb.fit(features, df_train.iloc[0:3500].label)
y_pred = gnb.predict(test_features)
print("Number of mislabeled points out of a total %d points : %d" % (test_features.shape[0], (df_test.iloc[:3500].label != y_pred).sum()))

Number of mislabeled points out of a total 3500 points : 148


In [234]:
y_pred = gnb.predict(valid_features)
print("Number of mislabeled points out of a total %d points : %d" % (valid_features.shape[0], (df_valid.iloc[:3500].label != y_pred).sum()))

Number of mislabeled points out of a total 2258 points : 149


We also train the model with the **spam2** dataset to compare the predictions.

In [289]:
test_sample = df2
test_features = cv.transform(test_sample.text).toarray()

gnb.score(test_features, test_sample.label)

0.7040684115055714

In [275]:
y_pred = gnb.predict(test_features)

In [49]:
df_train.iloc[0]

text             Free shipping on all jewelry and accessories o...
label                                                         spam
contains_link                                                False
Name: 0, dtype: object

#### Testen mit Daten von GPT3

In [324]:
gpt_features = cv.transform(df_gpt.text).toarray()
gnb.score(gpt_features, df_gpt.label)

0.5789473684210527

In [325]:
y_pred = gnb.predict(gpt_features)

In [328]:
df_gpt["outcome"] = df_gpt.label == y_pred

In [329]:
df_gpt

,text,label,outcome
0,Dear Name n know could earning money TODAY We ...,spam,False
1,Dear customer selected participate exclusive o...,spam,False
2,Hey customer amazing new product absolutely ne...,spam,False
3,Dear friend selected get free luxury watch Cli...,spam,True
4,Hey special offer right want miss Click link f...,spam,False
5,Hi noticed missing tons amazing opportunities ...,spam,False
6,n Valued Customer exclusive offer Claim free g...,spam,True
7,Greetings chosen receive Amazing offer Visit w...,spam,True
8,Hey proud owner brand new Rolex watch without ...,spam,True
9,Hey friend dreams come true lifetime offer Get...,spam,True


In [51]:
from sklearn.naive_bayes import GaussianNB
from sklearn.datasets import load_iris

# Load the iris dataset
iris = load_iris()
X = iris['data']
y = iris['target']

# Create a GaussianNB classifier
gnb = GaussianNB()

# Fit the classifier to the data
gnb.fit(X, y)

# Get the predicted probabilities for the first sample
probs = gnb.predict_proba([X[0]])[0]

# Print the predicted probabilities
print(probs)

[1.00000000e+00 1.35784265e-18 7.11283512e-26]


# Lineare Regression

Text-Analyse mit Linearer Regression

In [132]:
cv = CountVectorizer()
features = cv.fit_transform(df_train.iloc[0:2000].text).toarray()
test_features = cv.transform(df_test.iloc[0:2000].text).toarray()

In [134]:
def label_to_numerical(label):
    if label == "spam":
        return 0
    if label == "normal":
        return 1
    else:
        return np.NAN

label = df_train.iloc[:2000].label.apply(label_to_numerical)
test_label = df_test.iloc[:2000].label.apply(label_to_numerical)

In [135]:
reg = LinearRegression().fit(features, label)

In [144]:
reg.score(test_features, test_label)

-3.0893262944415882

In [149]:
reg = LogisticRegression().fit(features, label)
reg.score(test_features, test_label)

c:\Users\Tim\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.992

In [192]:
reg.score(test_features, test_label)

0.14

Lineare Regression ohne Text-Analyse

In [150]:
def label_to_numerical(label):
    if label == "spam":
        return 0
    if label == "normal":
        return 1
    else:
        return np.NAN

data = df_train[["contains_link", "contains_ip", "length"]].iloc[:2000]
test_data = df_test[["contains_link", "contains_ip", "length"]].iloc[:2000]
label = df_train.iloc[:2000].label.apply(label_to_numerical)
test_label = df_test.iloc[:2000].label.apply(label_to_numerical)
data["prediction"] =  reg.predict(features)
test_data["prediction"] = reg.predict(test_features)

In [151]:
reg2 = LinearRegression().fit(data, label)

In [152]:
reg2.score(test_data, test_label)

0.628608423279085

In [126]:
data
data["prediction"] = pred

In [ ]:
reg2.score(features, label)

In [ ]:
Logistic_reg = LogisticRegression().fit(data, label)

Test mit alten Daten

In [184]:
test_features = cv.transform(df.iloc[0:2000].text).toarray()

In [188]:
def label_to_numerical(label):
    if label == "spam":
        return 0
    if label == "normal":
        return 1
    else:
        return np.NAN

test_label = df.iloc[:2000].label.apply(label_to_numerical)

In [189]:

Logistic_reg.score(test_data, test_label)

0.149

### Decision Tree Classifier

In [84]:
cv = CountVectorizer()
features = cv.fit_transform(df_train.text)
test_features = cv.transform(df_test.text)
spam2_features = cv.transform((df.text))

In [98]:
DcsTree = DecisionTreeClassifier()
DcsTree.fit(features, df_train.label)

DecisionTreeClassifier()

In [99]:
DcsTree.score(test_features, df_test.label)

0.9953333333333333

In [100]:
DcsTree.score(spam2_features, df.label)

0.15466666666666667

### Random Forest

In [88]:
cv = CountVectorizer()
features = cv.fit_transform(df_train.text)
test_features = cv.transform(df_test.text)
spam2_features = cv.transform((df.text))

In [89]:
RndFrst = RandomForestClassifier()
RndFrst.fit(features, df_train.label)

RandomForestClassifier()

##### Test mit selben Dataset

In [90]:
RndFrst.score(test_features, df_test.label)

0.9913333333333333

##### Test mit anderem Dataset

In [91]:
RndFrst.score(spam2_features, df.label)

0.15466666666666667

### RandomForest mit Gridsearch Fine-Tunen

In [92]:
# Define the parameters you want to search over
param_grid = {'max_depth': [1, 2, 3, 4, 5],
              'min_samples_leaf': [1, 2, 3, 4, 5],
              'splitter': ["best", "random"]}

# Create a decision tree model
model = DecisionTreeClassifier()

# Create the grid search object
grid_search = GridSearchCV(model, param_grid, cv=5)

# Fit the grid search object to the training data
grid_search.fit(features, df_train.label)

# Print the best hyperparameters found by the grid search
print(grid_search.best_params_)

{'max_depth': 1, 'min_samples_leaf': 1, 'splitter': 'best'}


In [101]:
DcsTree = DecisionTreeClassifier(max_depth = 1, min_samples_leaf = 1, splitter = 'best')
DcsTree.fit(features, df_train.label)
print(f"Score mit Testdaten aus dem selben Dataset: {DcsTree.score(test_features, df_test.label)}")
print(f"Score mit Testdaten aus dem alten Dataset: {DcsTree.score(spam2_features, df.label)}")

Score mit Testdaten aus dem selben Dataset: 0.994
Score mit Testdaten aus dem alten Dataset: 0.15466666666666667


In [103]:
# Define the parameters you want to search over
param_grid = {
                'max_depth': [1, 2, 3, 4, 5], 
                'min_samples_leaf': [1, 2, 3, 4, 5],}

# Create a decision tree model
model = RandomForestClassifier()

# Create the grid search object
grid_search = GridSearchCV(model, param_grid, cv=5)

# Fit the grid search object to the training data
grid_search.fit(features, df_train.label)

# Print the best hyperparameters found by the grid search
print(grid_search.best_params_)

{'max_depth': 1, 'min_samples_leaf': 1}


In [104]:
RndFrst = RandomForestClassifier(max_depth = 1, min_samples_leaf = 1)
RndFrst.fit(features, df_train.label)
print(f"Score mit Testdaten aus dem selben Dataset: {RndFrst.score(test_features, df_test.label)}")
print(f"Score mit Testdaten aus dem alten Dataset: {RndFrst.score(spam2_features, df.label)}")

Score mit Testdaten aus dem selben Dataset: 0.986
Score mit Testdaten aus dem alten Dataset: 0.15466666666666667
